In [4]:
from ib_async import *
ib = IB()
util.startLoop() # Only for jupyter
ib.connect(port=4004,clientId=0)


<IB connected to 127.0.0.1:4004 clientId=0>

### View Your Positions

In [5]:
ib.positions()

[Position(account='DU7351505', contract=Option(conId=695095758, symbol='TSLA', lastTradeDateOrContractMonth='20240524', strike=165.0, right='C', multiplier='100', currency='USD', localSymbol='TSLA  240524C00165000', tradingClass='TSLA'), position=4.0, avgCost=1267.07185),
 Position(account='DU7351505', contract=Option(conId=695072875, symbol='RIVN', lastTradeDateOrContractMonth='20240524', strike=8.5, right='C', multiplier='100', currency='USD', localSymbol='RIVN  240524C00008500', tradingClass='RIVN'), position=5.0, avgCost=257.69686),
 Position(account='DU7351505', contract=Stock(conId=76792991, symbol='TSLA', exchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), position=600.0, avgCost=175.04833335)]

In [6]:
df_positions=util.df(ib.positions())

In [12]:
df_positions['symbol']=df_positions.contract.apply(lambda l:l.symbol)
df_positions['contract_type']=df_positions.contract.apply(lambda l:l.secType)
df_positions['expiry']=df_positions.contract.apply(lambda l:l.lastTradeDateOrContractMonth if l.secType=='OPT' else None)
df_positions['strike']=df_positions.contract.apply(lambda l:l.strike if l.secType == 'OPT' else None)
df_positions

,account,contract,position,avgCost,symbol,contract_type,expiry,strike
0,DU7351505,"Option(conId=695095758, symbol='TSLA', lastTra...",4.0,1267.071850,TSLA,OPT,20240524,165.0
1,DU7351505,"Option(conId=695072875, symbol='RIVN', lastTra...",5.0,257.696860,RIVN,OPT,20240524,8.5
2,DU7351505,"Stock(conId=76792991, symbol='TSLA', exchange=...",600.0,175.048333,TSLA,STK,None,NaN


### Account Info/Summary

In [13]:
ib.accountSummary()

[AccountValue(account='DU7351505', tag='AccountType', value='TRUST', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='Cushion', value='1', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='DayTradesRemaining', value='-1', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='DayTradesRemainingT+1', value='-1', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='DayTradesRemainingT+2', value='-1', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='DayTradesRemainingT+3', value='-1', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='DayTradesRemainingT+4', value='-1', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='LookAheadNextChange', value='1715952600', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='AccruedCash', value='-2080.80', currency='USD', modelCode=''),
 AccountValue(account='DU7351505', tag='AvailableFunds', value='898230.35', currency

In [14]:
ib.accountValues()

[AccountValue(account='DU7351505', tag='AccountCode', value='DU7351505', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='AccountOrGroup', value='DU7351505', currency='BASE', modelCode=''),
 AccountValue(account='DU7351505', tag='AccountOrGroup', value='DU7351505', currency='ILS', modelCode=''),
 AccountValue(account='DU7351505', tag='AccountOrGroup', value='DU7351505', currency='USD', modelCode=''),
 AccountValue(account='DU7351505', tag='AccountReady', value='true', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='AccountType', value='TRUST', currency='', modelCode=''),
 AccountValue(account='DU7351505', tag='AccruedCash', value='-2080.80', currency='BASE', modelCode=''),
 AccountValue(account='DU7351505', tag='AccruedCash', value='0.00', currency='ILS', modelCode=''),
 AccountValue(account='DU7351505', tag='AccruedCash', value='-2080.80', currency='USD', modelCode=''),
 AccountValue(account='DU7351505', tag='AccruedCash-C', value='0.00', currenc

In [17]:
df_account_summary=util.df(ib.accountSummary())
df_account_summary.head()

,account,tag,value,currency,modelCode
0,DU7351505,AccountType,TRUST,,
1,DU7351505,Cushion,1,,
2,DU7351505,DayTradesRemaining,-1,,
3,DU7351505,DayTradesRemainingT+1,-1,,
4,DU7351505,DayTradesRemainingT+2,-1,,


In [20]:
df_account_summary[df_account_summary['tag'].str.contains('netLiquidation',case=False)]

,account,tag,value,currency,modelCode
24,DU7351505,NetLiquidation,961809.92,USD,
38,All,NetLiquidationByCurrency,724560.00,ILS,
63,All,NetLiquidationByCurrency,766859.2164,USD,
88,All,NetLiquidationByCurrency,961809.9195,BASE,


### Market Data
* Need a few subscriptions!
* US Equity and Options Add-On Streaming Bundle (NP) - Trader Workstation
* US Securities Snapshot and Futures Value Bundle (NP,L1) - Trader Workstation ( Waived if monthly commissions reach 30 USD )

In [22]:
contract = Stock(symbol='TSLA',exchange='SMART',currency='USD')
contract

Stock(symbol='TSLA', exchange='SMART', currency='USD')

In [23]:
ib.qualifyContracts(contract)
contract

Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS')

### Get historical 5 minute data

In [24]:
bars=ib.reqHistoricalData(contract=contract,endDateTime='',durationStr='3 D',barSizeSetting='5 mins',
                     whatToShow='TRADES',useRTH=False,keepUpToDate=True)

In [32]:
util.df(bars)

,date,open,high,low,close,volume,average,barCount
0,2024-05-15 04:00:00-04:00,178.08,178.20,177.90,178.16,9636.0,178.032000,48
1,2024-05-15 04:05:00-04:00,178.16,178.20,178.00,178.20,6539.0,178.153000,39
2,2024-05-15 04:10:00-04:00,178.15,178.20,178.00,178.20,6800.0,178.131000,24
3,2024-05-15 04:15:00-04:00,178.20,178.23,178.06,178.10,8400.0,178.176000,35
4,2024-05-15 04:20:00-04:00,178.19,178.20,178.10,178.20,2388.0,178.173000,14
...,...,...,...,...,...,...,...,...
419,2024-05-17 06:55:00-04:00,174.72,174.84,174.72,174.84,2843.0,174.766000,8
420,2024-05-17 07:00:00-04:00,174.84,175.29,174.75,175.10,11397.0,174.993000,66
421,2024-05-17 07:05:00-04:00,175.13,175.15,175.03,175.03,2872.0,175.101000,13
422,2024-05-17 07:10:00-04:00,175.03,175.09,174.89,175.09,2387.0,175.029000,14


### ORDERING ( DO NOT RUN!!!)

In [31]:
limit_order = LimitOrder(action='BUY',totalQuantity=100,lmtPrice=10)

In [33]:
trade = ib.placeOrder(contract,limit_order)

In [37]:
trade.log

[TradeLogEntry(time=datetime.datetime(2024, 5, 17, 11, 18, 29, 545449, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 5, 17, 11, 18, 30, 282764, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)]

In [34]:
trade.orderStatus

OrderStatus(orderId=4588, status='PreSubmitted', filled=0.0, remaining=100.0, avgFillPrice=0.0, permId=1842923052, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0)

In [35]:
ib.openOrders()

[LimitOrder(orderId=4588, permId=1842923052, action='BUY', totalQuantity=100.0, lmtPrice=10.0, auxPrice=0.0)]

In [36]:
ib.openTrades()

[Trade(contract=Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), order=LimitOrder(orderId=4588, permId=1842923052, action='BUY', totalQuantity=100.0, lmtPrice=10.0, auxPrice=0.0), orderStatus=OrderStatus(orderId=4588, status='PreSubmitted', filled=0.0, remaining=100.0, avgFillPrice=0.0, permId=1842923052, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 5, 17, 11, 18, 29, 545449, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 5, 17, 11, 18, 30, 282764, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError='')]

#### Modify Order

In [39]:
order = trade.order
order.lmtPrice=200
trade = ib.placeOrder(contract,order)

In [40]:
trade.orderStatus

OrderStatus(orderId=4588, status='PreSubmitted', filled=0.0, remaining=100.0, avgFillPrice=0.0, permId=1842923052, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0)

##### Order didn't fill because we are premarket and didn't specify outsideRTH=True

### Lets cancel the orders

In [41]:
for order in ib.openOrders():
    ib.cancelOrder(order)

In [42]:
ib.openOrders()

[]

#### But we are premarket so specify outsideRth=True

In [48]:
limit_order = LimitOrder(action='BUY',totalQuantity=100,lmtPrice=176,outsideRth=True)

In [49]:
trade = ib.placeOrder(contract,limit_order)

In [50]:
trade.orderStatus

OrderStatus(orderId=4590, status='Filled', filled=100.0, remaining=0.0, avgFillPrice=175.16, permId=1842923054, parentId=0, lastFillPrice=175.16, clientId=0, whyHeld='', mktCapPrice=0.0)

### Look at fills

In [51]:
ib.fills()

[Fill(contract=Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), execution=Execution(execId='00025b46.6646d784.01.01', time=datetime.datetime(2024, 5, 17, 11, 22, 45, tzinfo=datetime.timezone.utc), acctNumber='DU7351505', exchange='ISLAND', side='BOT', shares=100.0, price=175.16, permId=1842923054, clientId=0, orderId=4590, liquidation=0, cumQty=100.0, avgPrice=175.16, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2, pendingPriceRevision=False), commissionReport=CommissionReport(execId='00025b46.6646d784.01.01', commission=1.0, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 5, 17, 11, 22, 46, 410116, tzinfo=datetime.timezone.utc))]

In [53]:
df_fills=util.df(ib.fills())

In [59]:
util.df(df_fills.execution.tolist())

,execId,time,acctNumber,exchange,side,shares,price,permId,clientId,orderId,liquidation,cumQty,avgPrice,orderRef,evRule,evMultiplier,modelCode,lastLiquidity,pendingPriceRevision
0,00025b46.6646d784.01.01,2024-05-17 11:22:45+00:00,DU7351505,ISLAND,BOT,100.0,175.16,1842923054,0,4590,0,100.0,175.16,,,0.0,,2,False


In [61]:
util.df(df_fills.commissionReport.tolist())

,execId,commission,currency,realizedPNL,yield_,yieldRedemptionDate
0,00025b46.6646d784.01.01,1.0,USD,0.0,0.0,0
